# Library

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

# Data

In [15]:
X_Data = np.loadtxt("X_Training.txt", dtype=np.float32)
Y_Data = np.loadtxt("Y_Training.txt", dtype=np.float32)

Features = 9 # Information: Frame, Track ID, 3D Dimensions (H, W, L), 3D Location (X,Y,Z), Rotation Y
X_Data = X_Data.reshape(X_Data.shape[0], X_Data.shape[1] // Features, Features)

X_Data = torch.tensor(X_Data)
Y_Data = torch.tensor(Y_Data)

print(X_Data.shape, type(X_Data))
print(Y_Data.shape, type(Y_Data))
print('-'.center(60,'-'))
# Information: 3D Dimensions (H, W, L), 3D Location (X,Y,Z), Rotation Y
X_Data = X_Data[:,:,2:9]
Y_Data = Y_Data[:,2:9]

print(X_Data.shape)
print(Y_Data.shape)

# Train Data
print('-'.center(60,'-'))
batch_size = 133
lote = 1

XT_Data = X_Data.reshape(int(batch_size/lote),lote,5,7)
YT_Data = Y_Data.reshape(int(batch_size/lote),lote,7)
print(XT_Data.shape)
print(YT_Data.shape)

X_train = XT_Data[:100]
X_test  = XT_Data[101:]
y_train = YT_Data[:100]
y_test  = YT_Data[101:]

torch.Size([133, 5, 9]) <class 'torch.Tensor'>
torch.Size([133, 9]) <class 'torch.Tensor'>
------------------------------------------------------------
torch.Size([133, 5, 7])
torch.Size([133, 7])
------------------------------------------------------------
torch.Size([133, 1, 5, 7])
torch.Size([133, 1, 7])


# Model

In [5]:
class RNNModel(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        
        super(RNNModel, self).__init__()
        
        self.hidden_dim = hidden_dim #Hidden Dimension
        self.layer_dim = layer_dim # Number of hidden layers

        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu') #Building the RNN       
        self.fc = nn.Linear(hidden_dim, output_dim) # Readout layer
        
    def forward(self, x):
       
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:,-1,:])
        
        return out

## Parameters


In [7]:
input_dim = 7
hidden_dim = 150
layer_dim = 1
output_dim = 7
batch_size = 25
seq_dim = 5

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

criterion = nn.MSELoss()

learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Load Model

In [12]:
PATH = "model.pth"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# model.eval()
# # - or -
# model.train()



In [56]:
pp = Y_Data


pp = pp.reshape(133,1,7)
print(pp.size()[0])

133


In [54]:
model.eval()
out4 = model(pp)
out4.shape

torch.Size([133, 7])

## Test model

In [71]:
def validation(model, X_test, y_test, criterion):
    
    test_loss = 0
    accuracy = 0

    for inputs, labels in zip(X_test, y_test):
        
        inputs = inputs
        output = model.forward(inputs)
        test_loss += criterion(output, labels).item()
        print('-'.center(80,'-'))
        
        print('x-1: ',inputs[0][4])        
        print('y: ',labels)
        print('x: ',output)
        
        ps = torch.exp(output)
        
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean() # IoU

    return test_loss, accuracy

In [72]:
model.eval()
test_loss, accuracy = validation(model,X_test, y_test,criterion)
print('Test loss: ',  test_loss)
print('Accuracy: ',  accuracy)

--------------------------------------------------------------------------------
x-1:  tensor([ 1.5960,  1.6981,  3.5626,  2.3154,  1.5452, 10.0725, -1.6301])
y:  tensor([[ 1.5960,  1.6981,  3.5626,  2.4311,  1.5646,  9.3605, -1.6179]])
x:  tensor([[ 1.5596,  1.5776,  3.6180,  2.5095,  1.5932,  9.4397, -1.6428]],
       grad_fn=<AddmmBackward>)
--------------------------------------------------------------------------------
x-1:  tensor([ 1.5000,  1.6750,  4.0267, -6.6190,  1.7619, 20.2679,  1.4115])
y:  tensor([[ 1.5000,  1.6750,  4.0267, -7.0051,  1.7655, 19.6339,  1.3933]])
x:  tensor([[ 1.6049,  1.6089,  4.1472, -6.7411,  1.9087, 19.5274,  1.4149]],
       grad_fn=<AddmmBackward>)
--------------------------------------------------------------------------------
x-1:  tensor([ 1.5000,  1.6750,  4.0267, -7.8050,  1.9514, 16.8605,  1.3595])
y:  tensor([[ 1.5000,  1.6750,  4.0267, -7.8807,  1.9966, 16.2139,  1.3511]])
x:  tensor([[ 1.6738,  1.6136,  4.1745, -7.7488,  1.9191, 16.1336,  1